# Codes for living park dataset preproc

In [1]:
# from pathlib import Path
import sys
from pathlib import Path
import glob
import os
import shutil
import numpy as np
import pandas as pd

# system PATH
code_path_str='/scratch/mr_proc'
sys.path.append(code_path_str)

# study PATH
code_dir = Path(code_path_str)
dataset_name = "PPMI"
dataset_path = Path('/scratch/0tmp/'+dataset_name)
dataset_out_path    = Path('/scratch/0tmp/'+dataset_name+'_SessionOrganized')

dataset_out_df_path = Path(code_dir / 'tab_data'  / 'PPMI_livingpark_metadata.csv')

In [2]:
# read meta data
import xml.etree.ElementTree as ET

files = [f for f in os.listdir(dataset_path) if f[-3:]=='xml']
tree = ET.parse(str(dataset_path/files[0]))
root = tree.getroot()
subject_info_list= [];
for file in files:
    tree = ET.parse(str(dataset_path/file))
    root = tree.getroot()
    data_dict={};
    data_dict['Image Data ID']=root[0][3][4][6][0].text
    data_dict['Subject']=root[0][3][0].text
    data_dict['Group']=root[0][3][1].text
    data_dict['Sex']=root[0][3][2].text
    data_dict['Visit']=root[0][3][3][0].text
    data_dict['Age']=root[0][3][4][1].text
    data_dict['Modality']=root[0][3][4][5][1].text
    data_dict['Description']=root[0][3][4][6][1].text
    data_dict['Acq Date']=root[0][3][4][5][2].text
    subject_info_list.append(data_dict)
df=pd.DataFrame(subject_info_list)

In [3]:
# T1 sequence description
T1W_SERIES = [
    'MPRAGE 2 ADNI',
    'MPRAGE ADNI',
    'MPRAGE GRAPPA 2',
    'MPRAGE GRAPPA2',
    'MPRAGE GRAPPA2(adni)',
    'MPRAGE w/ GRAPPA',
    'MPRAGE_GRAPPA',
    'MPRAGE_GRAPPA_ADNI',
    'MPRAGE GRAPPA',
    'SAG T1 3D MPRAGE',
    'sag mprage',
    'MPRAGEadni',
    'MPRAGE GRAPPA_ND',
    '3D SAG',
    'MPRAGE T1 SAG',
    'MPRAGE SAG',
    'SAG T1 3DMPRAGE',
    'SAG T1 MPRAGE',
    'SAG 3D T1',
    'SAG MPRAGE GRAPPA2-NEW2016',
    'SAG MPRAGE GRAPPA_ND',
    'Sag MPRAGE GRAPPA',
    'AXIAL T1 3D MPRAGE',
    'SAG MPRAGE GRAPPA',
    'sT1W_3D_FFE',
    'sT1W_3D_ISO',
    'sT1W_3D_TFE',
    'sag 3D FSPGR BRAVO straight',
    'SAG T1 3D FSPGR',
    'SAG FSPGR 3D '
    'SAG 3D FSPGR BRAVO STRAIGHT',
    'SAG T1 3D FSPGR 3RD REPEAT',
    'SAG FSPGR BRAVO',
    'SAG SPGR 3D',
    'SAG 3D SPGR',
    'FSPGR 3D SAG',
    'SAG FSPGR 3D',
    'SAG 3D FSPGR BRAVO STRAIGHT',
    'SAG FSPGR 3D ',
    't1_mpr_ns_sag_p2_iso',
    'T1',
    'T1 Repeat',
    'AX T1',
    'axial spgr',
    'T1W_3D_FFE AX',
    # added by Vincent
    'AX T1 SE C+',
    '3D SAG T1 MPRAGE',
    '3D SAG T1 MPRAGE_ND',
    '3D T1',
    '3D T1 MPRAGE',
    '3D T1-weighted',
    'Accelerated Sag IR-FSPGR',
    'MPRAGE',
    'MPRAGE - Sag',
    'MPRAGE Phantom GRAPPA2',
    'MPRAGE w/ GRAPPA 2',
    'PPMI_MPRAGE_GRAPPA2',
    'SAG 3D T1 FSPGR',
    'SAG FSPGR 3D VOLUMETRIC T1',
    'Sag MPRAGE GRAPPA_ND',
    'T1-weighted, 3D VOLUMETRIC',
    'tra_T1_MPRAGE',
    '3D T1-weighted_ND', ## added from livingpark
    '3D T1 _weighted',
    'Sagittal 3D Accelerated MPRAGE',
    'T1 REPEAT',
    'MPRAGE Repeat',
    'SAG_3D_MPRAGE']

In [4]:
df=pd.DataFrame(subject_info_list)
def encode_visit(x_):
    if x_=='Baseline':
        return 1 # 
    elif x_=='Screening':
        return 0
    else:
        return -1
df['Visit'] = [ encode_visit(x) for x in df['Visit'] ]
df_baseline=df[df['Visit']>=0]
#df["T1 include"] = [1 if x in T1W_SERIES  else 0 for x in df['Description']]
#t1_df=df[df["T1 include"]==1].copy()
#t1_df=t1_df.drop(columns=['T1 include'])
df_baseline["Image Data ID"] = ["I"+str(x) for x in df_baseline['Image Data ID']]
#df=df.apply(lambda x: "\""+str(x)+"\"")
print('Final size: ', df.shape)
# save file
df_baseline.to_csv(dataset_out_df_path, index=False)

Final size:  (356, 9)


/tmp/ipykernel_224/1850272041.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_baseline["Image Data ID"] = ["I"+str(x) for x in df_baseline['Image Data ID']]


In [5]:
dataset_out_df_path = Path(code_dir / 'tab_data'  / 'PPMI_livingpark_metadata.csv')
ppmi_img_dl_data=pd.read_csv(dataset_out_df_path, dtype=str)
subj_="72419"
image_tab_df = ppmi_img_dl_data[ppmi_img_dl_data['Subject']==subj_].loc[:,['Image Data ID', 'Visit']].copy()
image_tab_df
img_ids=image_tab_df['Image Data ID'].unique()
image_tab_df[image_tab_df["Image Data ID"]=="I1519145"]
ppmi_img_dl_data[ppmi_img_dl_data['Subject']==subj_]

,Image Data ID,Subject,Group,Sex,Visit,Age,Modality,Description,Acq Date
337,I945538,72419,Prodromal,F,1,59.16,MRI,T1 Repeat,2017-10-06
338,I945537,72419,Prodromal,F,1,59.16,MRI,T1,2017-10-06
339,I945535,72419,Prodromal,F,1,59.16,MRI,T2,2017-10-06
340,I945536,72419,Prodromal,F,1,59.16,MRI,T2,2017-10-06


In [6]:
print(len(files),ppmi_img_dl_data.shape)

356 (341, 9)
